# Vehicle Service Tracker and Cost Estimator

In [ ]:
import datetime
import math
import sys
import os
import re
import pdfplumber
import pytesseract
from PIL import Image

# Creating service checklist with intervals & costs in Australia
service_guide = {
    "Checking the engine oil & oil filter": {"km": 10000, "cost": 150},
    "Inspecting brake pads": {"km": 20000, "cost": 300},
    "Checking coolant flush": {"km": 50000, "cost": 100},
    "Checking transmission fluid": {"km": 80000, "cost": 200},
    "Inspecting timing belt": {"km": 100000, "cost": 500},
    "Tire replacement": {"km": 60000, "cost": 400}
}

FILENAME = "service_history.txt" #where the service history will stored.

def get_user_data():
    try:
        last_service_date = input("Enter the last service date (YYYY-MM-DD): ")
        last_service_date = datetime.datetime.strptime(last_service_date, "%Y-%m-%d")
        present_mileage = int(input("Enter the current mileage (km): "))
        last_service_mileage = int(input("Enter the mileage at last service (km): "))
        avg_km_perday = float(input("Approximately how far does your car travel daily (km): "))
        return last_service_date, present_mileage, last_service_mileage, avg_km_perday
    except ValueError:
        print("Invalid input! Please enter data in correct format.")
        sys.exit(1)

def save_service_history(data):
    with open(FILENAME, "a") as file:
        file.write(data + "\n")

def read_service_history():
    if os.path.exists(FILENAME):
        print("\n  Service History")
        with open(FILENAME, "r") as file:
            print(file.read())
    else:
        print("\nNo previous service records found!")

def wear_and_cost(present_mileage, last_service_mileage, avg_km_perday):
    km_after_last_service = present_mileage - last_service_mileage
    
    if avg_km_perday == 0:
        print("\nAverage km per day cannot be zero.")
        return
    
    print("\nVehicle Service Status & Cost Estimation")
    estimated_cost = 0
    service_log = f"Service Date: {datetime.datetime.today().strftime('%Y-%m-%d')}\n" #today's date
    
    for service, data in service_guide.items():
        km = data["km"]
        cost = data["cost"]
        
        if km_after_last_service >= km:
            print(f"{service} is DUE!! Estimated Cost: ${cost}")
            estimated_cost += cost
            service_log += f"{service}: DUE (Cost: ${cost})\n"
        else:
            remaining_km = km - km_after_last_service
            estimated_time = remaining_km / avg_km_perday
            estimated_date = datetime.datetime.today() + datetime.timedelta(days=estimated_time)
            print(f"{service} is OK. Next service in {remaining_km} km (~{estimated_time:.1f} days, due on {estimated_date.strftime('%Y-%m-%d')}).")
            service_log += f"{service}: Next due on {estimated_date.strftime('%Y-%m-%d')} ({remaining_km} km left)\n"
    
    print(f"\nTotal Estimated Service Cost: ${estimated_cost}")
    
    if estimated_cost > 1000:
        print("Your service cost is high!! Consider budgeting accordingly.")
    
    save_service_history(service_log)

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"Error processing PDF: {e}")
        return None
    return text.strip()

def analyze_service_report(text):
    text = text.lower()
    recommended_services = []
    
    service_checklist = {
        "oil": "Checking the engine oil & oil filter",
        "brake": "Inspecting brake pads",
        "coolant": "Checking coolant flush",
        "transmission": "Checking transmission fluid",
        "belt": "Inspecting timing belt",
        "tire": "Tire replacement"
    }
    
    for keyword, service in service_checklist.items():
        if re.search(rf"\b{keyword}\b", text):
            recommended_services.append(service)
    
    if recommended_services:
        print("\nWe recommend you check the following based on your last service report:")
        for service in recommended_services:
            print(f"- {service}")
    else:
        print("\nNo additional services were found from the report.")

def upload_service_report():
    file_path = input("\nPlease enter the file path to your service report (PDF): ")
    
    if not os.path.exists(file_path):
        print("Error! File not found. Please enter a valid path.")
        return
    
    extracted_text = extract_text_from_pdf(file_path)
    
    if extracted_text:
        print("\nRetrieved Service Report Content:\n")
        print(extracted_text)
        analyze_service_report(extracted_text)

def main():
    print("Vehicle Service Checklist & Cost Calculator")
    read_service_history()
    
    choice = input("\nDo you want to enter data manually (press 1) or upload a service report PDF (press 2)? ")
    
    if choice == "1":
        last_service_date, present_mileage, last_service_mileage, avg_km_perday = get_user_data()
        wear_and_cost(present_mileage, last_service_mileage, avg_km_perday)
    elif choice == "2":
        upload_service_report()
    else:
        print("Invalid choice! Please enter 1 or 2.")
        sys.exit(1)

if __name__ == "__main__":
    main()

Vehicle Service Checklist & Cost Calculator

  Service History
Service Date: 2025-03-29
Checking the engine oil & oil filter: DUE (Cost: $150)
Inspecting brake pads: DUE (Cost: $300)
Checking coolant flush: DUE (Cost: $100)
Checking transmission fluid: Next due on 2029-08-15 (24000 km left)
Inspecting timing belt: Next due on 2033-04-09 (44000 km left)
Tire replacement: Next due on 2025-12-20 (4000 km left)


